In [ ]:
import json

# 先建立 dict 格式的 account.json 檔，並取得其中的帳密資料
file = open("account.json", "r", encoding="utf-8")
strJson = file.read()
dictJson = json.loads(strJson)
print(dictJson)

In [9]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re


'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

'''
自訂變數
'''
# 想爬取資料的來源網址
url = 'https://www.instagram.com/tommychen1986/?hl=zh-tw'

# 放置爬取的資料
listData = []

# 放置 ig 每個格子裡面的超連結
listLink = []

# 設定 set 物件，放置影片、圖片連結，協助過濾重複的元素
setTmp = set()

def login():
    try:
        # 前往首頁
        driver.get('https://www.instagram.com');

        # 等待互動元素出現 (這裡用帳號文字欄位)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, 'input[name="username"]') 
            )
        )

        # 輸入帳號
        driver.find_element(By.CSS_SELECTOR, 'input[name="username"]').send_keys(dictJson['username'])

        # 輸入密碼
        driver.find_element(By.CSS_SELECTOR, 'input[name="password"]').send_keys(dictJson['password'])

        # 強制等待
        sleep(2)

        # 按下登入
        driver.find_element(By.CSS_SELECTOR, 'button[type="submit"].sqdOP.L3NKy.y3zKF').click()
        
        # 強制等待
        sleep(2)
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        sleep(3)
        driver.quit()
        
# 滾動頁面
def scroll():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, "div.Nnq7C.weEfm") 
            )
        )

        '''
        totalHeight => 瀏覽器內部的高度
        offset => 當前捲動的量(高度)
        count => 計無效滾動次數
        limit => 最大無效滾動次數
        wait_second => 每次滾動後的強制等待時間
        '''
        totalHeight = 0
        offset = 0
        count = 0
        limit = 3
        wait_second = 3

        # 在捲動到沒有元素動態產生前，持續捲動
        while count <= limit:
            # 每次移動高度
            offset = driver.execute_script(
                'return window.document.documentElement.scrollHeight;'
            )

            '''
            或是每次只滾動一點距離，
            以免有些網站會在移動長距離後，
            將先前移動當中的元素隱藏
            offset += 600
            '''

            # 捲動的 js code
            js_code = f'''
                window.scrollTo({{
                    top: {offset}, 
                    behavior: 'smooth' 
                }});
            '''

            # 執行 js code
            driver.execute_script(js_code)

            # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
            sleep(wait_second)

            # 透過執行 js 語法來取得捲動後的當前總高度
            totalHeight = driver.execute_script(
                'return window.document.documentElement.scrollHeight;'
            );

            # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
            if offset >= totalHeight:
                count += 1
                
            # 為了實驗功能，捲動超過一定的距離，就結束程式
            if offset >= 600:
                break

            
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        sleep(10)
        driver.quit()           
               
# 走訪頁面
def visit():
    # 前往指定連結
    driver.get(url);
    
def getUrl():
    # 取得主要元素的集合
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'div.Nnq7C.weEfm div.v1Nh3.kIKUG._bz0w a')
    
    # 逐一檢視元素
    for index, a in enumerate(a_elms):
        # 測試功能，所以只選三個連結
        if index > 4:
            break
        
        # 取得圖片連結
        aLink = a.get_attribute('href')
        ##print("取得網址: {}".format(aLink))
        
        # 放資料到 list 中
        listLink.append(aLink)
        # print(listLink)
        
def parse():
    for aLink in listLink:
        # 清空暫存圖片、影片連結的 set
        setTmp.clear()
        
        # 走訪頁面
        driver.get(aLink)
        
       #full_match = re.search(regex, aLink)
        
        '''
        取得頁面的 id：
            full_match = re.search(regex, aLink)[0]
            group_1 = re.search(regex, aLink)[1]
        
        範例：
            matchObj[0]  : /p/CGpaFHrHV2j/
            matchObj[1]  : CGpaFHrHV2j 
        '''
        # 取得 ig 連結的 id
        regex = r'\/p\/([a-zA-Z0-9-_]+)\/'
        pageId = re.search(regex, aLink)[1]
        
        ##print("網頁 ID: {}".format(pageId))
        
        # 強制等待
        sleep(2)
        
        '''
        判斷是否有「向右」按鈕：
            若有，則代表會有多個 li；
            若無，則代表只有一個 li

        備註：
            由於 find_elements 回傳 list 格式，
            所以可以用 len() 來取得元素長度，
            判斷元素是否存在
        '''     
        # 當有右箭頭可以按的時候，代表頁面中同時有多張圖片或影片，甚至混合
        if len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
            # 取得多元素資訊
            while len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
                # 按下向右按鈕
                driver.find_element(By.CSS_SELECTOR, "button._6CZji").click()

                '''將元素裡面的屬性值(例如 src)，視為 value 加入 set 當中'''
                li_elms = driver.find_elements(By.CSS_SELECTOR, "li.Ckrof")

                # 檢視當前現有的 li (解決滑動時，前面一部分或後面一部分 li.Ckrof 會消失的問題)
                for li in li_elms:
                    if len(li.find_elements(By.CSS_SELECTOR, "article img.FFVAD")) > 0: # 如果這個 li 裡面有 img
                        # 取得 img 連結
                        imgSrc = li.find_element(By.CSS_SELECTOR, "article img.FFVAD").get_attribute('src')

                        # 加入圖片連結到 set 中
                        setTmp.add(imgSrc)
                        ##會跑出每張圖片的網址
                        ##print(imgSrc) 
                    else:
                        pass

                # 強制等待
                sleep(1)
        else:
            # 當沒有右箭頭可以按的時候，代表是獨立的單張圖片或影片
            if len( driver.find_elements(By.CSS_SELECTOR, "article img.FFVAD") ) > 0: # 若是 img
                # 取得 img
                imgSrc = driver.find_element(By.CSS_SELECTOR, "article img.FFVAD").get_attribute('src')

                # 加入圖片連結到 set 中
                setTmp.add(imgSrc)
            else:
                pass

        # 整合此次網頁連結的元素資料
        listData.append({
            "id": pageId,
            "url": aLink,
            "listDL": list(setTmp)
        })  
    ##會同時跑出每個listData內的資訊  
    ##print(listData) 
    
def savejson():
    #"w"是寫入模式
    fp = open("igTommyChenTest.json","w",encoding="utf-8")
    fp.write(json.dumps(listData, ensure_ascii=False, indent=4))
    fp.close()
    
def download():
    fp = open("igTommyChenTest.json","r",encoding="utf-8")
    strJson = fp.read()
    #fp要記得關比較省資源?
    fp.close()
    
    # 如果無此資料夾就建立
    folderPath = 'igTommyChenTest'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    
     # 將 json 轉成 list 
    listResult = json.loads(strJson)
    
    for dictObj in listResult:
        for dl_url in dictObj['listDL']:
            print("下載連結:{}".format(dl_url))
            strFileName = re.search(r'https?:\/\/\S+.\/(\S+\.(jpe?g|mp4))', dl_url)[1]
            os.system('curl "{}" -o ./{}/{}'.format(dl_url, folderPath, strFileName))
            
            ## 無法下載  os.system(f"curl {dl_url} -o ./{folderPath}/{strFileName}")
        
           ##os.system(f"curl {obj['link']} -o {folderPath}/{obj['id']}.png")
    
    
    # 關閉瀏覽器
def close():
    driver.quit()

if __name__ == '__main__':
   # login()
   # visit()
   # getUrl()
  #  parse()
    savejson()
    download()
    close()    

下載連結:https://instagram.ftpe7-2.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/244169349_557838098776175_4222060484747688775_n.jpg?_nc_ht=instagram.ftpe7-2.fna.fbcdn.net&_nc_cat=111&_nc_ohc=a-aDOoJHkDkAX8d9n4k&tn=38ANpVDZKfkBeAlr&edm=AABBvjUBAAAA&ccb=7-4&oh=31850a0ff142d5e7a18bbe420c38b681&oe=616618D3&_nc_sid=83d603
下載連結:https://instagram.ftpe7-4.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/243663883_564174038190775_8059652532052399032_n.jpg?_nc_ht=instagram.ftpe7-4.fna.fbcdn.net&_nc_cat=107&_nc_ohc=H2_oyjzl3ukAX-tiqYd&edm=AABBvjUBAAAA&ccb=7-4&oh=6307449a572049bebe7ed2496c438523&oe=616620C1&_nc_sid=83d603
下載連結:https://instagram.ftpe7-2.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/243506913_184364323836741_7811623539543787693_n.jpg?_nc_ht=instagram.ftpe7-2.fna.fbcdn.net&_nc_cat=109&_nc_ohc=VLHmjuSW72YAX8Qw49C&edm=AABBvjUBAAAA&ccb=7-4&oh=b54fcb7e2b33e9ae0d445ce7b81c7b7d&oe=6166660B&_nc_sid=83d603
下載連結:https://instagram.ftpe7-3.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/243879447_3058857987767438_

下載連結:https://instagram.ftpe7-3.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/234674409_565324977848414_3567063964339709253_n.jpg?_nc_ht=instagram.ftpe7-3.fna.fbcdn.net&_nc_cat=102&_nc_ohc=Qv4LA59lsu4AX8p0chU&edm=AABBvjUBAAAA&ccb=7-4&oh=a5aabd09bf9cd400728b7cf7069e4c87&oe=6165B078&_nc_sid=83d603
